The purpose of this notebook is to explore the process_file function to understand what exactly we want it to do.

In [1]:
import re
import sys

In [2]:
with open('fbiucr2016/data/2016 SHR FIle LIst.txt') as f:
    lines = f.readlines()

In [3]:
DATA_LINE_REGEX = re.compile('^[\s\d]\s*\d\d')
PARTIAL_LINE_REGEX = re.compile('^\s{37}\s*\d*')

Create example of different types of data lines

In [4]:
full = lines[5-1]
partial_o = lines[22-1]
partial_v = lines[98-1]
text = lines[1-1]

In [6]:
full

'    01 AL00112 3 01  A    001   C    33  M  W      16  M  B      11    ST   99       HOOVER                    ALA\n'

In [7]:
partial_o

'                                                   00  M  W      12    ST   81   A\n'

In [8]:
partial_v

'                                     50  M  W  N\n'

In [9]:
text

'1                                SUPPLEMENTARY HOMICIDE REPORT - FILE LISTING 2016                             UCR44300    07/07/17\n'

Text does not return a match object

In [15]:
DATA_LINE_REGEX.search(full)

<_sre.SRE_Match object; span=(0, 6), match='    01'>

In [16]:
DATA_LINE_REGEX.search(partial_o)

<_sre.SRE_Match object; span=(0, 53), match='                                                 >

In [17]:
DATA_LINE_REGEX.search(partial_v)

<_sre.SRE_Match object; span=(0, 39), match='                                     50'>

In [18]:
DATA_LINE_REGEX.search(text)

Test second conditional on remaining three types. Only returns match for partial.

In [20]:
PARTIAL_LINE_REGEX.search(full)

In [21]:
PARTIAL_LINE_REGEX.search(partial_o)

<_sre.SRE_Match object; span=(0, 53), match='                                                 >

In [22]:
PARTIAL_LINE_REGEX.search(partial_v)

<_sre.SRE_Match object; span=(0, 39), match='                                     50'>

##### Full line path

In [25]:
fields = full.split("\s*")
fields

['    01 AL00112 3 01  A    001   C    33  M  W      16  M  B      11    ST   99       HOOVER                    ALA\n']

I don't believe this result is what you are looking for. Are you looking for a list of field strings like this instead? 

*mpg*: Correct, this should use the RE to split it into fields.  Stripping before splitting should create the correct output.

In [27]:
fields2 = re.split("\s*", full)
fields2

/Users/laurencewarner/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['',
 '01',
 'AL00112',
 '3',
 '01',
 'A',
 '001',
 'C',
 '33',
 'M',
 'W',
 '16',
 'M',
 'B',
 '11',
 'ST',
 '99',
 'HOOVER',
 'ALA',
 '']

Do we want to strip the line first, to avoid the two blank fields?

In [33]:
fields2 = re.split("\s*", full.strip())
fields2

/Users/laurencewarner/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


['01',
 'AL00112',
 '3',
 '01',
 'A',
 '001',
 'C',
 '33',
 'M',
 'W',
 '16',
 'M',
 'B',
 '11',
 'ST',
 '99',
 'HOOVER',
 'ALA']

And then do we want to write this list to a file? As in output_file.write("-> " + line.strip())

Why do we want to concatenate with the arrow "->"?

*mpg*: Was for testing.  The goal should be:
- Generate a unique ID for each event
- Add the missing fields to each line from the corresponding event (including event ID) so each line is a complete record
  - We could instead normalize this by creating three tables, linked by the event ID: event, victim, offender.  THis would be the "right" thing to do, but the easy thing to do right now is to just fill in the missing data.  We can transform into a normal form down the line if we want

##### Partial line path

In [34]:
partial_o.strip()

'00  M  W      12    ST   81   A'

Do you want to just write this to the output file, or do you also want to split this into fields?

##### Return
We could return the output txt file.